In [1]:
import os
import CityGraph
import folium
import geopandas as gpd
import pandas as pd

In [2]:
cg = CityGraph.CityGraph('data/sp_network.gpickle',40.0)

Loading file...
Preprocessing city mesh...
Refining mesh...
Building tree...


In [3]:
cg.load_polygon_mesh('data/malha-sp-setores-2010.kml','Name',40.0)

Refining mesh...


True

In [4]:
cg.load_polygon_csv_data('data/sectorsindices.csv','Cod_setor')

True

In [5]:
cg.load_crimes_layer('data/crimes_ssp.csv','LATITUDE','LONGITUDE')

True

In [6]:
cg.compute_layer_crimes(initial_date = pd.to_datetime('01-01-2020'), final_date = pd.to_datetime('31-12-2020'))

/var/folders/n6/5pm6t1f133z900vqfnq_xtjc0000gn/T/ipykernel_89167/650629977.py:1: UserWarning: Parsing '31-12-2020' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cg.compute_layer_crimes(initial_date = pd.to_datetime('01-01-2020'), final_date = pd.to_datetime('31-12-2020'))


True

In [7]:
cg.layer_crimes_count

,crime_mobile,crime_vehicle,crime_all
0,3.0,1.0,4.0
1,4.0,0.0,4.0
2,0.0,0.0,0.0
3,1.0,0.0,1.0
4,2.0,0.0,2.0
...,...,...,...
119743,0.0,0.0,0.0
119744,0.0,0.0,0.0
119745,3.0,0.0,3.0
119746,1.0,0.0,1.0


In [8]:
cg.load_layer_points('data/SAD69-96_SHP_pontoonibus.shp',23,'K',True,-45.0,-45.0)

True

In [9]:
cg.load_layer_points('data/SAD69-96_SHP_estacaometro_point.shp',23,'K')

True

In [10]:
cg.load_layer_points('data/SAD69-96_SHP_estacaotrem_point.shp',23,'K')

True

In [11]:
cg.load_layer_points('data/sad6996_terminal_onibus.shp',23,'K')

True

In [12]:
cg.load_layer_mesh('data/AGSN_2019.shp')

0

In [13]:
cg.load_station_measurements('Barueri','data/Barueri.CSV')

True

In [14]:
cg.load_station_measurements('Interlagos','data/Interlagos.CSV')

True

In [15]:
cg.load_station_measurements('Mirante','data/Mirante.CSV')

True

In [16]:
cg.load_measu_temp_agg_funcs('data/Sparse_temp_agg_funcs.CSV')

True

In [17]:
cg.load_station_info('data/Stations_info.CSV')

True

In [18]:
cg.compute_layer_weather('01-01-2020','31-12-2020')

True

In [19]:
cg.save_preprocessed_CityGraph('data/sp-sibgrapi2022.bin')

In [20]:
cg.compute_feature_vector(0)

/Users/thalesvieira/Projects/sibgrapi2022/CityGraph.py:1181: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  feature_vec = pd.Series([])
/Users/thalesvieira/Projects/sibgrapi2022/CityGraph.py:1192: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_vec = feature_vec.append(self.layer_crimes_count.loc[vert_ind])
/Users/thalesvieira/Projects/sibgrapi2022/CityGraph.py:1206: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_vec = feature_vec.append(ibge_list)
/Users/thalesvieira/Projects/sibgrapi2022/CityGraph.py:1216: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_vec = feature_vec.append(transpor

crime_mobile                            3.0
crime_vehicle                           1.0
crime_all                               4.0
Renda_media_por_domicilio       5160.404542
Renda_media_responsaveis        3015.097073
Responsaveis_sem_renda_taxa        0.093952
Alfabetizados_de_7_a_15_anos            1.0
menores_de_18_anos_taxa            0.185002
18_a_65_anos_taxa                  0.698416
maiores_de_65_anos_taxa            0.116582
Pontos_de_onibus                        2.0
Estacao_de_metro                        0.0
Estacao_de_trem                         0.0
Terminal_de_onibus                      0.0
Favela_proxima                          0.0
Precipitacao_Total              1372.100297
Temperatura_Maxima                37.557652
Temperatura_Minima                 7.558559
dtype: object

In [ ]:
import folium
import osmnx as ox
import networkx as nx

lat = -23.549090314066356
long = -46.65458954194309

rectangle_center = [lat, long]

result=cg.query_point_in_graph_radius_in_city_mesh(lat,long,0.7)

In [ ]:
m = folium.Map(location=[lat, long], zoom_start=15, tiles='CartoDB positron')

nearest=cg.query_point_in_city_mesh(lat, long, True)
print(nearest)
folium.CircleMarker(rectangle_center, radius=5, color='blue').add_to(m)
folium.CircleMarker(cg.city_vert_list[nearest], radius=8, color='pink').add_to(m)

for vert in result:
    folium.CircleMarker(cg.city_vert_list[vert], radius=5, color='red').add_to(m)  
    
    
rectangle_side = 0.01

for u,v,a in cg.city_mesh_graph.edges(data=True):
    vertA = cg.city_vert_list[cg.city_vert_nxind_to_ind_dict[u]]
    vertB = cg.city_vert_list[cg.city_vert_nxind_to_ind_dict[v]]  
    if ((abs(vertA[0]-rectangle_center[0])<rectangle_side and abs(vertA[1]-rectangle_center[1])<rectangle_side) or
        (abs(vertB[0]-rectangle_center[0])<rectangle_side and abs(vertB[1]-rectangle_center[1])<rectangle_side)):
            folium.PolyLine([vertA,vertB], color="yellow", weight=2.5, opacity=1).add_to(m)
            folium.CircleMarker(vertA, radius=2, color='blue').add_to(m)  
            folium.CircleMarker(vertB, radius=2, color='blue').add_to(m)    

m      